In [1]:
## Import packages
import numpy as np
import pandas as pd
import math
import random
from scipy.stats import poisson
from scipy.special import binom
import seaborn as sns

## Setup problem definition

## Test case

- T = 4h
- d = 5 min
- beta = 9 minutes service time (exponential)
- N = 24 patients
- I = T / d # the number of intervals

In [2]:
class schedule:
  def __init__(self, beta: int, T: int, d: int, N: int):
    self.beta = beta # = 1/μ : average service time
    self.T = T # number of intervals
    self.d = d # length of interval
    self.N = N # total number of patients
    self.x:list[int] = np.repeat(0, self.T) # schedule with x[t] as number of 
                                            # patients scheduled at the start of
                                            # interval t, t = 1,...,T; initially
                                            # set at zero
  def reset_schedule(self):
    for t in range(self.T):
      self.x[t] = 0
  def make_random_schedule(self, min_x: int, max_x: int, step: int=1):
    n = self.N
    for i in range(0,self.T, step):
      if n >= 0:
        r = random.randint(min_x, max_x)
        self.x[i] = min(r,n)
        n = n - r
        print(n, self.x[i])
      else:
        return
  def make_initial_schedule(self):
    for i in range(self.N):
      t = round(i*self.T / self.N)
      print(i)
      if i > self.T:
        i-= 1
      self.x[t] += 1

# beta = 20
# T = 48
# d = 5
# N = 10
# s1 = schedule(beta,T,d,N)
# step = math.floor(beta/d)
# s1.make_random_schedule(0,2, step)
# print(s1.x)
# print(f'total number of patients in schedule {sum(s1.x)}')
# s1.reset_schedule()
# print(s1.x)
# s1.make_initial_schedule()
# print(s1.x)
# print(f'total number of patients in schedule {sum(s1.x)}')

## Functions

In [3]:
def recursive_sum(i):
  if i==1:
    return 1
  return i+recursive_sum(i-1)

recursive_sum(5)

15

In [4]:
#not used because of recursion depth limit in Python
#def factorial(i):
#  if i==1:
#    return 1
#  return i * factorial(i-1)
#
#factorial(5)

for i in range(10):
  print(math.factorial(i))

1
1
2
6
24
120
720
5040
40320
362880


In [5]:
# Not necessary - use scipy poisson.pmf instead
def service(k, beta):
  return pow(beta,k)*math.exp(-beta) / math.factorial(k)

service(10,20)

0.005816306518345136

In [6]:
# Not necessary - use scipy binom instead
#def binomial_coef(n,k):
#  return factorial(n)/(factorial(k)*factorial(n-k))

#print(f'{binomial_coef(4,1)} {binomial_coef(4,2)} {binomial_coef(4,3)}')

In [7]:
print(f'{binom(4,1)} {binom(4,2)} {binom(4,3)}')

4.0 6.0 4.0


In [8]:
# moving patients iteratively, local search
def iteration_small(x, p, beta, precision, precision_limit, n, no_show, I, d, alpha_W, alpha_I, alpha_T, results): # name of function should be meaningful
  x_new = x.copy()
  for m in range(0, I-1):
    for k in list(range(0, I)): # [::-1]
      if x_new[k] > 0: # Adding one vector is equivalent to moving the arrival of one patient from interval k to interval k-1
        x_new[k] -= 1
        # x_new[((k + m - 1) % I) + 1] += 1
        x_new[(k + m + 1) % I] += 1 # x_new[(k + m - 1) % I] += 1
        results_temp = calcResults(x_new, p, beta, precision, precision_limit, n, no_show, I, d, 0, alpha_W, alpha_I, alpha_T) # messy x, p, beta, precision, n, no_show, I, d, eind, alpha_W, alpha_I, alpha_T
        print(x_new, results_temp)
        if results_temp['objVal'] < results['objVal']: # super costly to recalculate probabilities every iteration
          return x_new, results_temp
        else: 
          x_new[k] += 1
          # x_new[((k + m - 1) % I) + 1] -= 1 # store index as variable?
          x_new[(k + m + 1) % I] -= 1 # x_new[(k + m - 1) % I] -= 1
  return x, None

In [9]:
def limit(mu): # name of function should be meaningful
  return int(max(mu+4*math.sqrt(mu),100))

# limit(1/beta)

In [10]:
#def create_a1(beta, precision=0.9999): #Poisson P(potential departures = k)
#  k = 0
#  a=[]
#  while sum(a) < precision:
#    a.append(service(k, beta)) # Service function not necessary - use scipy poisson instead
#    k+=1
#  return(a)

#a1=create_a(10)
#a1

In [11]:
# def create_p(beta, precision=0.9999): #Poisson P(potential departures = k)
#   k = 0
#   p = []
#   while sum(p) < precision:
#     p.append(service(k, beta))
#     k+=1
#   return(p)

# p = create_p(9)
# p

In [12]:
def create_p(beta, size, precision=0.9999): #Poisson P(potential departures = k)
  k = 0
  p = []
  while sum(p) < precision:
    p.append(poisson.pmf(k, beta))
    k+=1
  while len(p) < size:
    p.append(0)
  return p, k

In [13]:
# p_dist = np.array([range(len(p)), p])
# p_dist = pd.DataFrame({'i':p_dist[0,:], 'p':p_dist[1,:]})
# p_dist

In [14]:
# data = p_dist[p_dist['p']>0]
# sns.barplot(data=data, x="i", y="p")

In [15]:
# def create_p(beta, precision=0.9999): #Poisson P(potential departures = k)
#   k = 0
#   p = []
#   while sum(p) < precision:
#     p.append(poisson.pmf(k=k, mu=beta))
#     k+=1
#   return(p)

# p = create_p(9)
# p

In [16]:
def calcExponentialLimit(mu):
  return int(max(mu+4*mu**0.5,100))

In [17]:
def binomCoeff(k, i):
  return math.factorial(k) / (math.factorial(k - i) * math.factorial(i))

def binomPMF(k, i, m, add_v, no_show):
  return binomCoeff(k, m) * add_v[m][i] * (1 - no_show)**m * no_show**(k-m)

In [18]:
# two main components
# find a schedule to evaluate <- efficient search procedure to still end up with a good schedule
# evaluate a specific schedule <- as fast a possible
def calculateProbabilities(x, p, beta, precision, precision_limit, n, no_show, I, d):
  # count = len(p)
  count = precision_limit
  waiting = 0
  idletime = 0
  tardiness = 0
  p_plus = np.zeros((1000,1000))
  p_min = np.zeros((1000,1000))
  v = np.zeros((1000,1000))
  w = np.zeros((1000,1000,1000))
  add_v = np.zeros((1000,1000)) # needs a predetermined size

#	v[k][i]=probability of having i units of work given that 
#	k patients are scheduled for the interval.
#	p[i]= probability of serving the patient in i mins given that
#	the average service time is beta.
  add_v[0][0] = 1
  for k in range(n+1):
    limit = calcExponentialLimit(beta*k)
    # print(f'limit={limit}')
    i = 0
    sum_v = 0
    while sum_v < precision and i <= limit:
      z = 0
      while n < count:
        add_v[k][i] += p[z] * add_v[k-1][i-n]
        z += 1
      sum_v += add_v[k][i] 
      i += 1

  for k in range(n+1): 
    i = 0
    sum_v = 0
    while sum_v < precision and i <= limit:
      for m in range(k):
        v[k][i] += binomPMF(k, i, m, add_v, no_show)
      sum_v += v[k][i]
      i += 1

  # Constraint 2
  p_min[0][0] = 1
  sum_p = 0
  i = 1
  while sum_p < precision and i <= limit:
    p_plus[0][i] = v[x[0]][i]
    sum_p += p_plus[0][i]
    i += 1

  for t in range(1, I):
    # Constraint 3
    for k in range(d):
      p_min[t][0] += p_plus[t-1][k]

  for t in range(1, I):
    # Constraint 4
    for i in range(1,limit):
      p_min[t][i] = p_plus[t-1][i+d]

  for t in range(1, I):
    # Constraint 5
    if t != I+1:
      for i in range(limit):
        for j in range(i):
          p_plus[t][i] += p_min[t][j] * v[x[t]][i-j]

  # Tardiness calcs
  for k in range(limit):
    tardiness += k * p_min[I][k] # I+1

  # print('p_min:', p_min[I]) # tardiness always 0 atm

  # Idle time calcs new array of given shape and type, filled with zeros.
  idletime = (I * d) + tardiness - (n * (1 - no_show) * beta) # I-1?

  # Waiting time calcs
  for t in range(0, I):
    if x[t] > 0:
      for k in range(limit):
        w[t][0][k] = p_min[t][k] # w[t][1][k]
    if x[t] > 1:
      for i in range(1, x[t]+1): # range(2, x[t]+1)
        for k in range(limit+1):
          # temp_p = poisson.pmf(-np.arange(0,k+1)+k, mu=beta) # calculating all at once is far faster
          for j in range(k+1):
            w[t][i][k] += w[t][i-1][j] * p[k-j] # limit max 100 so k can be > 23 which is length of p, error when j=0 k=23
  for t in range(0, I):
    for i in range(0, x[t]):
      for k in range(limit+1):
        waiting += w[t][i][k] * k
  waiting /= n

  return p_min, waiting, idletime, tardiness

In [19]:
def calcFracExcess(p_min, I):
  fracExcess = 0
  t = I+1
  for j in range(1, len(p_min[t])):
    fracExcess += p_min[t][j]
  fracExcess *= 100
  return fracExcess

In [20]:
def calcResults(x, p, beta, precision, precision_limit, n, no_show, I, d, eind, alpha_W, alpha_I, alpha_T):
  p_min, waitingTime, idleTime, tardiness = calculateProbabilities(x, p, beta, precision, precision_limit, n, no_show, I, d)
  objVal = alpha_W*waitingTime + alpha_I*idleTime + alpha_T*tardiness
  results = {'p_min' : p_min, 'waitingTime' : waitingTime, 'idleTime' : idleTime, 'tardiness' : tardiness, 'objVal' : objVal}
  
  if eind == 1:
    fracExcess = calcFracExcess(p_min, I)
    results['fracExcess'] = fracExcess
  
  return results

In [21]:
# precision = 0.9999
# n = 5
# beta = 5
# T = 60
# d = 5
# I = int(T/d)
# s = schedule(beta,T,d,n)
# s.make_initial_schedule()
# x = s.x
# p = create_a2(beta, precision)
# no_show = 0
# result = calculateProbabilities(x, p, beta, precision, n, no_show, I)

In [22]:
# print(f'p_min: {result[0]}\n waiting: {result[1]}\n idletime: {result[2]}\n tardiness: {result[3]}')

In [23]:
precision = 0.9999
n = 50
beta = 9
T = 4*60 # total time
d = 5 # interval size
I = int(T/d) # number of intervals
s = schedule(beta,T,d,n)
s.make_initial_schedule()
x = s.x
size = calcExponentialLimit(beta*n)+1 # size of p has to be at least as big as the limit value here
print(size)
p, precision_limit = create_p(beta, size, precision)
no_show = 0
#x = list(np.zeros(I, dtype=int))
#x[0] = n
iend = 0
# alpha_I = 0.2
# alpha_T = 0.4 - 0.00000001 # patient doctor centric slider
# alpha_W = (1 - alpha_T) + 0.00000001 - 0.2
alpha_I = 0.2
alpha_T = 0.4 # patient doctor centric slider
alpha_W = 0.4
res = calcResults(x, p, beta, precision, precision_limit, n, no_show, I, d, iend, alpha_W, alpha_I, alpha_T)
print(x)
print(res)
while res['objVal'] > 0:
  x, res = iteration_small(x, p, beta, precision, precision_limit, n, no_show, I, d, alpha_W, alpha_I, alpha_T, res)
print(x)
print(res)
# p = create_a2(beta, precision)
# no_show = 0
# result = calculateProbabilities(x, p, beta, precision, n, no_show, I)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
535
[1 0 0 0 0 1 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0
 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0
 0 0 0 1 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 1
 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0
 0 1 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 1 0 0
 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1
 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0]
{'p_min': array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), 'waitingTime': 0.0, 'idleTime': -210.0, 'tardiness': 0.0, 'objVal': -42.0}
[1 0 0 0 0 1 0 0 0 0 1 0 0 0 1 0 0 0 0 1 